<a href="https://colab.research.google.com/github/Rozieyati/Data-Science-Project/blob/main/STQD6134_GroupA_Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

STQD6134 : Business Analytics Project 1 - Group A

Scenario:
You are a Business Analyst at Streamify, a digital streaming company that offers on-demand video content through monthly and yearly subscriptions.

Problem Statement:

1. To  understand how different customer segments and subscription plans affect revenue and customer retention

2. To perform an analysis on subscription data from the last year and provide insights on:

  *   Subscription and cancellation trends
  *   Revenue performance by plan type and region
  *   Customer engagement metrics

# 1. Data Processing

In [ ]:
#data simulation, to generate dataset with sample size 2000
set.seed(15)
n <- 2000

#Attributes
CustomerID <- paste0("C", sprintf("%04d", 1:n))
JoinDate <- sample(seq(as.Date('2024-01-01'),as.Date('2024-12-31'), by="day"), n, replace = TRUE)
ActiveMonths <- sample(1:12, n, replace = TRUE)
library(dplyr)
CancelDate <- if_else(runif(n) < 0.25, JoinDate + ActiveMonths*30, as.Date(NA))   # 25% cancellations
Region <- sample(c("North", "South", "East", "West"), n, replace = TRUE)
SubscriptionType <- sample(c("Basic", "Standard", "Premium"), n, replace = TRUE, prob=c(0.4, 0.35, 0.25))
MonthlyFee <- ifelse(SubscriptionType == "Basic", 10,
                     ifelse(SubscriptionType == "Standard", 20, 30))
TotalStreams <- round(rnorm(n, mean=170, sd=60))
TotalStreams <- pmax(TotalStreams,0)  #to ensure there is no negative value,
DeviceType <- sample(c("Mobile", "Smart TV", "Laptop", "Tablet"), n, replace = TRUE)
PaymentMethod <- sample(c("Card", "Online Wallet", "NetBanking"), n, replace = TRUE)
stream_data <- data.frame(CustomerID, JoinDate, CancelDate, Region, SubscriptionType, MonthlyFee,
                          ActiveMonths, TotalStreams, DeviceType, PaymentMethod)
stream_data$Revenue <- stream_data$MonthlyFee * stream_data$ActiveMonths
head(stream_data)
write.csv(stream_data, "stream_data.csv", row.names = FALSE)  #simulated dataset
getwd()




Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




,CustomerID,JoinDate,CancelDate,Region,SubscriptionType,MonthlyFee,ActiveMonths,TotalStreams,DeviceType,PaymentMethod,Revenue
,<chr>,<date>,<date>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>,<chr>,<dbl>
1,C0001,2024-02-06,NA,East,Basic,10,5,116,Smart TV,Card,50
2,C0002,2024-12-27,NA,West,Standard,20,8,206,Mobile,Card,160
3,C0003,2024-06-10,NA,North,Basic,10,5,227,Smart TV,Card,50
4,C0004,2024-10-20,2025-07-17,South,Basic,10,9,185,Laptop,Card,90
5,C0005,2024-06-25,NA,West,Premium,30,6,252,Smart TV,Online Wallet,180
6,C0006,2024-09-17,NA,East,Premium,30,6,103,Mobile,Online Wallet,180


[1] "/content"

In [ ]:
# Importing CSV using read.csv
data1 <- read.csv("stream_data.csv", header=TRUE, stringsAsFactors=TRUE)
head(data1)
str(data1)
summary(data1)

#Task 1 - Preprocessing

#check & handle missing value
na_counts <- colSums(is.na(data1))
print(na_counts) #check no. of missing value in each attribute

data1$CancelDate <- as.Date(as.character(data1$CancelDate))
data1$CancelDate[is.na(data1$CancelDate)] <- as.Date("2030-12-31") #replace NA with future date #change data type to Date
na_counts <- colSums(is.na(data1))
print(na_counts) #check no. of missing value


#convert data type
str(data1)
data1$JoinDate <- as.Date(data1$JoinDate, origin = "1970-01-01")  #change the data type to Date


#create new variables
data1$IsActive <- data1$CancelDate == as.Date("2030-12-31")  #TRUE if CancelDate=NA which have been replaced by FutureDate
library(lubridate)
data1$MonthJoined <- month(data1$JoinDate, label = TRUE, abbr = TRUE)  #return Months in Mmm format in order.

#head(data1)
str(data1)

#Task 2 - Business Metric Calculations
#Total Revenue

#Average Revenue per User (ARPU)

#Revenue by Subscription Type

#Churn Rate - % of customers who cancelled during the year.

#Regional Revenue - Total revenue by region.

#Average Engagement (Streams per Active Month) - Average number of videos watched per month by customers.

#Monthly Join Trend - Number of new customers joining per month.

#Device Usage Breakdown - Most common devices used

#Task 3 - Visualization
# Revenue by Subscription Type
# Revenue by Region
# Monthly Join Trend
# Device Usage
# TotalStreams distribution (to show engagement)


,CustomerID,JoinDate,CancelDate,Region,SubscriptionType,MonthlyFee,ActiveMonths,TotalStreams,DeviceType,PaymentMethod,Revenue
,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<fct>,<fct>,<int>
1,C0001,2024-02-06,NA,East,Basic,10,5,116,Smart TV,Card,50
2,C0002,2024-12-27,NA,West,Standard,20,8,206,Mobile,Card,160
3,C0003,2024-06-10,NA,North,Basic,10,5,227,Smart TV,Card,50
4,C0004,2024-10-20,2025-07-17,South,Basic,10,9,185,Laptop,Card,90
5,C0005,2024-06-25,NA,West,Premium,30,6,252,Smart TV,Online Wallet,180
6,C0006,2024-09-17,NA,East,Premium,30,6,103,Mobile,Online Wallet,180


'data.frame':	2000 obs. of  11 variables:
 $ CustomerID      : Factor w/ 2000 levels "C0001","C0002",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ JoinDate        : Factor w/ 364 levels "2024-01-01","2024-01-02",..: 37 360 161 293 176 260 216 203 361 83 ...
 $ CancelDate      : Factor w/ 346 levels "2024-02-03","2024-02-09",..: NA NA NA 297 NA NA NA 261 NA NA ...
 $ Region          : Factor w/ 4 levels "East","North",..: 1 4 2 3 4 1 4 2 3 3 ...
 $ SubscriptionType: Factor w/ 3 levels "Basic","Premium",..: 1 3 1 1 2 2 1 1 3 1 ...
 $ MonthlyFee      : int  10 20 10 10 30 30 10 10 20 10 ...
 $ ActiveMonths    : int  5 8 5 9 6 6 10 10 3 7 ...
 $ TotalStreams    : int  116 206 227 185 252 103 255 168 127 188 ...
 $ DeviceType      : Factor w/ 4 levels "Laptop","Mobile",..: 3 2 3 1 3 2 3 3 3 1 ...
 $ PaymentMethod   : Factor w/ 3 levels "Card","NetBanking",..: 1 1 1 1 3 3 3 2 1 3 ...
 $ Revenue         : int  50 160 50 90 180 180 100 100 60 70 ...


   CustomerID         JoinDate         CancelDate     Region   
 C0001  :   1   2024-06-24:  11   2024-09-03:   6   East :505  
 C0002  :   1   2024-07-06:  11   2025-05-29:   5   North:518  
 C0003  :   1   2024-10-01:  11   2024-09-19:   4   South:473  
 C0004  :   1   2024-03-04:  10   2024-10-27:   4   West :504  
 C0005  :   1   2024-03-15:  10   2024-11-25:   4              
 C0006  :   1   2024-03-23:  10   (Other)   : 478              
 (Other):1994   (Other)   :1937   NA's      :1499              
 SubscriptionType   MonthlyFee     ActiveMonths     TotalStreams  
 Basic   :784     Min.   :10.00   Min.   : 1.000   Min.   :  0.0  
 Premium :540     1st Qu.:10.00   1st Qu.: 3.000   1st Qu.:130.0  
 Standard:676     Median :20.00   Median : 6.000   Median :171.0  
                  Mean   :18.78   Mean   : 6.421   Mean   :169.8  
                  3rd Qu.:30.00   3rd Qu.: 9.000   3rd Qu.:212.0  
                  Max.   :30.00   Max.   :12.000   Max.   :390.0  
                   

      CustomerID         JoinDate       CancelDate           Region 
               0                0             1499                0 
SubscriptionType       MonthlyFee     ActiveMonths     TotalStreams 
               0                0                0                0 
      DeviceType    PaymentMethod          Revenue 
               0                0                0 
      CustomerID         JoinDate       CancelDate           Region 
               0                0                0                0 
SubscriptionType       MonthlyFee     ActiveMonths     TotalStreams 
               0                0                0                0 
      DeviceType    PaymentMethod          Revenue 
               0                0                0 
'data.frame':	2000 obs. of  11 variables:
 $ CustomerID      : Factor w/ 2000 levels "C0001","C0002",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ JoinDate        : Factor w/ 364 levels "2024-01-01","2024-01-02",..: 37 360 161 293 176 260 216 203 361 83 


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




'data.frame':	2000 obs. of  13 variables:
 $ CustomerID      : Factor w/ 2000 levels "C0001","C0002",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ JoinDate        : Date, format: "2024-02-06" "2024-12-27" ...
 $ CancelDate      : Date, format: "2030-12-31" "2030-12-31" ...
 $ Region          : Factor w/ 4 levels "East","North",..: 1 4 2 3 4 1 4 2 3 3 ...
 $ SubscriptionType: Factor w/ 3 levels "Basic","Premium",..: 1 3 1 1 2 2 1 1 3 1 ...
 $ MonthlyFee      : int  10 20 10 10 30 30 10 10 20 10 ...
 $ ActiveMonths    : int  5 8 5 9 6 6 10 10 3 7 ...
 $ TotalStreams    : int  116 206 227 185 252 103 255 168 127 188 ...
 $ DeviceType      : Factor w/ 4 levels "Laptop","Mobile",..: 3 2 3 1 3 2 3 3 3 1 ...
 $ PaymentMethod   : Factor w/ 3 levels "Card","NetBanking",..: 1 1 1 1 3 3 3 2 1 3 ...
 $ Revenue         : int  50 160 50 90 180 180 100 100 60 70 ...
 $ IsActive        : logi  TRUE TRUE TRUE FALSE TRUE TRUE ...
 $ MonthJoined     : Ord.factor w/ 12 levels "Jan"<"Feb"<"Mar"<..: 2 12 6 10 6 9 8 7 12 

# 2. Business Metric Calculations

In [ ]:
## (1.) Total Revenue - Total Revenue generated during the year.

TotalRevenue <- sum(stream_data$Revenue, na.rm = TRUE)
TotalRevenue

[1] 242910

In [ ]:
## (2.) Average Revenue per User (ARPU) – Average revenue per customer.

ARPU <- mean(stream_data$Revenue, na.rm = TRUE)
ARPU

[1] 121.455

In [ ]:
## (3.) Revenue by Subscription Type - Compare revenue generated by Basic,
## Standard, and Premium Plans.

RevenueByType <- aggregate(Revenue ~ SubscriptionType,
                           data = stream_data, sum)
RevenueByType

SubscriptionType,Revenue
<chr>,<dbl>
Basic,49360
Premium,106230
Standard,87320


In [ ]:
## (4.) Churn Rate - % of customers who cancelled during the year.

ChurnRate <- sum(!is.na(stream_data$CancelDate)) / nrow(stream_data)
ChurnRate

[1] 0.2505

In [ ]:
## (5.) Regional Revenue - Total revenue by region.

RegionalRevenue <- aggregate(Revenue ~ Region,
                             data = stream_data, sum)
RegionalRevenue

Region,Revenue
<chr>,<dbl>
East,62320
North,64010
South,56700
West,59880


In [ ]:
## (6.) Average Engagement (Streams per Active Month) - Average number of videos
## watched per month by customers.

stream_data$Engagement <- stream_data$TotalStreams / stream_data$ActiveMonths
AvgEngagement <- mean(stream_data$Engagement, na.rm = TRUE)
AvgEngagement

[1] 44.47467

In [ ]:
## (7.) Monthly Join Trend - Number of new customers joining per months.

stream_data$MonthJoined <- format(stream_data$JoinDate, "%m")
MonthlyJoinTrend <- table(stream_data$MonthJoined)
MonthlyJoinTrend


 01  02  03  04  05  06  07  08  09  10  11  12 
172 155 179 166 172 178 188 162 162 171 141 154 

In [ ]:
## (8.) Device Usage Breakdown - Most common devices used to access the
## platform.

DeviceUsage <- table(stream_data$DeviceType)
DeviceUsage


  Laptop   Mobile Smart TV   Tablet 
     515      495      496      494 